## Kantai Bert with Byte Level Byte pair Encoding Tokenizer

1. Obtaining the dataset

### 1. Obtaining the Dataset

In [1]:
# dataset of books by Immanuel Kant

!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/master/Chapter03/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 10.7M    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  1 10.7M    1  128k    0     0  75271      0  0:02:29  0:00:01  0:02:28 75415
 31 10.7M   31 3488k    0     0  1273k      0  0:00:08  0:00:02  0:00:06 1274k
100 10.7M  100 10.7M    0     0  3393k      0  0:00:03  0:00:03 --:--:-- 3397k


In [2]:
# !pip uninstall tensorflow
# !pip install transformers

In [3]:
# getting versions of transformers and tokenizers
!pip list | grep -E 'transformers|tokenizers'

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import tokenizers
import transformers

d:\After\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
%%time
# using bytelevelbpetokenizer

from pathlib import Path

paths = [str(x) for x in Path(".").glob("*.txt")]
print(paths)

tokenizer = tokenizers.ByteLevelBPETokenizer()
print(f'Tokenizer: {tokenizer}')

tokenizer.train(files=paths, vocab_size=50000, min_frequency=2, special_tokens=[
    "<start>",
    "<pad>",
    "<\s>",
    "<unknown>",
    "<mask>",
])

['kant.txt']
Tokenizer: Tokenizer(vocabulary_size=0, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)
CPU times: total: 12.5 s
Wall time: 1.25 s


In [7]:
import os

tokenizer_dir = 'Tokenizer'

# print(os.path.exists(tokenizer_dir))
if os.path.exists(tokenizer_dir):
    # print(os.path.listdir)
    pass

if not os.path.exists(tokenizer_dir):
    os.mkdir(tokenizer_dir)
    print("Dir made")
tokenizer.save_model('Tokenizer')

Dir made


['Tokenizer\\vocab.json', 'Tokenizer\\merges.txt']